In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import random

my_headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:84.0) Gecko/20100101 Firefox/84.0"}

## 40 constituencies

In [358]:
const_ctr = 9000001

list_dict = []

# the last of the 40 is at 47, not 40
for x in range (47):
    
    problem_list = [9000013, 9000024, 9000027, 9000028, 9000030, 9000032, 9000033]
    if const_ctr in problem_list:
        print(const_ctr)
        const_ctr = const_ctr + 1
        continue
    
    url = f'https://www.bbc.co.uk/news/election/2021/wales/constituencies/W0{const_ctr}'
    req = requests.get(url, headers = my_headers)
    soup = bs(req.content)
    
    # parse the soup for a constituency
    some_dict = {}
    ppv = []
    
    # name of const
    for a in soup.find_all('h1'):
        name = a.text
        some_dict['constituency'] = name
    
    # region
    rl = []
    for b in soup.find_all('span', class_ = 'ssrcss-y40dsz-TextWrapper eayu6g40'):
        rl.append(b.text)
    region = rl[1]
    some_dict['region'] = region

    # hold result
    for c in soup.find_all('span', class_ = 'ssrcss-1e8pkn-FlashBold e1t3dmp51'):
        result = c.text
    some_dict['result'] = result

    # voters turned out, percentage, change
    tolist = []
    for f in soup.find_all('span', class_ = 'ssrcss-9v20ri-StyledResult e1rrn48s8'):
        tolist.append(f.text)
    to = tolist[0]
    to_change = tolist[1]
    for g in soup.find_all('span', class_ = 'ssrcss-1cuz2pt-TurnoutValue e1rrn48s6'):
        to_perc = g.text
    some_dict['turnout'] = to   
    some_dict['turnout_perc'] = to_perc
    some_dict['turnout_change'] = to_change 

    # the results
    for m in soup.find_all('div', class_ = 'ssrcss-1nfrpyt-GridContainer ez70f8k0'):
        
        for n in m.find_all('span', class_ = 'ssrcss-14p55mc-Supertitle e1j83d2f2'):
            party = n.text
            ppv.append(party)

        for o in m.find_all('span', class_ = 'ssrcss-zbxkfk-Title e1j83d2f3'):
            person = o.text
            ppv.append(person)

        for p in m.find_all('span', class_ = 'ssrcss-ce9ddf-ResultValue e1k9l0jz2'):
            votes = p.text
            ppv.append(votes)

        some_dict['results'] = ppv

    
    list_dict.append(some_dict)
    
    time.sleep(random.randint(2, 5))
    const_ctr = const_ctr + 1 
    if const_ctr % 5 == 0:
        print(const_ctr)
    
    

9000005
9000010
9000013
9000015
9000020
9000024
9000027
9000028
9000030
9000030
9000032
9000033
9000035
9000040
9000045


In [359]:
len(list_dict)

40

In [367]:
list_dict[4]

{'constituency': 'Vale of Clwyd',
 'region': 'North Wales',
 'result': 'CON GAIN',
 'turnout': '56,622',
 'turnout_perc': '46%',
 'turnout_change': '+3.10',
 'results': ['Conservative, ',
  'Gareth Davies',
  '10,792',
  'Labour, ',
  'Jason McLellan',
  '10,426',
  'Plaid Cymru, ',
  'Glenn Swingler',
  '2,972',
  'Liberal Democrat, ',
  'Lisa Jane',
  '782',
  'Reform UK, ',
  'Peter Dain',
  '552',
  'Independent, ',
  'David Thomas',
  '529']}

## Build the dataframe

In [371]:
# how long do the number of candidates get?

for x in list_dict:
    print(int(len(x['results'])/3))
    
# the biggest nb of candidates is 12

5
6
6
8
6
7
7
8
7
8
6
7
5
5
7
8
7
9
7
9
9
8
8
8
9
11
10
6
8
8
7
7
8
9
9
12
7
7
8
11


In [386]:
# basic df

df1 = pd.DataFrame(list_dict)

df1[:5]

,constituency,region,result,turnout,turnout_perc,turnout_change,results
0,Ynys Mon,North Wales,PC HOLD,"53,554",52%,+1.80,"[Plaid Cymru, , Rhun ap Iorwerth, 15,506, Cons..."
1,Arfon,North Wales,PC HOLD,"42,710",51%,0.00,"[Plaid Cymru, , Sian Gwenllian, 13,760, Labour..."
2,Aberconwy,North Wales,CON HOLD,"45,436",52%,+2.80,"[Conservative, , Janet Finch-Saunders, 9,815, ..."
3,Clwyd West,North Wales,CON HOLD,"58,671",48%,+2.80,"[Conservative, , Darren Millar, 11,839, Labour..."
4,Vale of Clwyd,North Wales,CON GAIN,"56,622",46%,+3.10,"[Conservative, , Gareth Davies, 10,792, Labour..."


In [385]:
col_list = ['one_party', 'one_person', 'one_vote', 
            'two_party', 'two_person', 'two_vote',
            'three_party', 'three_person', 'three_vote',
            'four_party', 'four_person', 'four_vote',
            'five_party', 'five_person', 'five_vote',
            'six_party', 'six_person', 'six_vote',
            'seven_party', 'seven_person', 'seven_vote',
            'eight_party', 'eight_person', 'eight_vote',
            'nine_party', 'nine_person', 'nine_vote',
            'ten_party', 'ten_person', 'ten_vote',
            'eleven_party', 'eleven_person', 'eleven_vote',
            'twelve_party', 'twelve_person', 'twelve_vote',
           ]

# rows list initialization
rows = []
  
# appending rows
for d in list_dict:
    row = d['results']
    rows.append(row)
      
df3 = pd.DataFrame(rows)
  
df3.columns = col_list
df3

,one_party,one_person,one_vote,two_party,two_person,two_vote,three_party,three_person,three_vote,four_party,...,nine_vote,ten_party,ten_person,ten_vote,eleven_party,eleven_person,eleven_vote,twelve_party,twelve_person,twelve_vote
0,"Plaid Cymru,",Rhun ap Iorwerth,"15,506","Conservative,",Lyn Hudson,"5,689","Labour,",Sam Egelstaff,"5,300","Reform UK,",...,None,None,None,None,None,None,None,None,None,None
1,"Plaid Cymru,",Sian Gwenllian,"13,760","Labour,",Iwan Wyn Jones,"5,108","Conservative,",Tony Thomas,"1,806","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
2,"Conservative,",Janet Finch-Saunders,"9,815","Plaid Cymru,",Aaron Wynne,"6,479","Labour,",Dawn McGuinness,"5,971","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
3,"Conservative,",Darren Millar,"11,839","Labour,",Joshua Hurst,"8,154","Plaid Cymru,",Elin Walker Jones,"5,609","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
4,"Conservative,",Gareth Davies,"10,792","Labour,",Jason McLellan,"10,426","Plaid Cymru,",Glenn Swingler,"2,972","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
5,"Labour,",Hannah Blythyn,"12,846","Conservative,",Mark Isherwood,"9,135","Plaid Cymru,",Paul Rowlinson,"2,097","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
6,"Labour,",Jack Sargeant,"12,622","Conservative,",Abigail Mainon,"8,244","Plaid Cymru,",Jack Morris,"1,886","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
7,"Labour,",Lesley Griffiths,"8,452","Conservative,",Jeremy Kent,"7,102","Plaid Cymru,",Carrie Harper,"4,832","Abolish the Welsh Assembly Party,",...,None,None,None,None,None,None,None,None,None,None
8,"Labour,",Ken Skates,"10,448","Conservative,",Barbara Hughes,"7,535","Plaid Cymru,",Llyr Gruffydd,"4,094","Liberal Democrat,",...,None,None,None,None,None,None,None,None,None,None
9,"Plaid Cymru,",Mabon ap Gwynfor,"11,490","Conservative,",Charlie Evans,"4,394","Labour,",Cian Ireland,"3,702","Propel,",...,None,None,None,None,None,None,None,None,None,None


In [430]:
df = pd.concat([df1, df3], axis = 1)
df

,constituency,region,result,turnout,turnout_perc,turnout_change,results,one_party,one_person,one_vote,...,nine_vote,ten_party,ten_person,ten_vote,eleven_party,eleven_person,eleven_vote,twelve_party,twelve_person,twelve_vote
0,Ynys Mon,North Wales,PC HOLD,"53,554",52%,+1.80,"[Plaid Cymru, , Rhun ap Iorwerth, 15,506, Cons...","Plaid Cymru,",Rhun ap Iorwerth,"15,506",...,None,None,None,None,None,None,None,None,None,None
1,Arfon,North Wales,PC HOLD,"42,710",51%,0.00,"[Plaid Cymru, , Sian Gwenllian, 13,760, Labour...","Plaid Cymru,",Sian Gwenllian,"13,760",...,None,None,None,None,None,None,None,None,None,None
2,Aberconwy,North Wales,CON HOLD,"45,436",52%,+2.80,"[Conservative, , Janet Finch-Saunders, 9,815, ...","Conservative,",Janet Finch-Saunders,"9,815",...,None,None,None,None,None,None,None,None,None,None
3,Clwyd West,North Wales,CON HOLD,"58,671",48%,+2.80,"[Conservative, , Darren Millar, 11,839, Labour...","Conservative,",Darren Millar,"11,839",...,None,None,None,None,None,None,None,None,None,None
4,Vale of Clwyd,North Wales,CON GAIN,"56,622",46%,+3.10,"[Conservative, , Gareth Davies, 10,792, Labour...","Conservative,",Gareth Davies,"10,792",...,None,None,None,None,None,None,None,None,None,None
5,Delyn,North Wales,LAB HOLD,"56,489",47%,+3.50,"[Labour, , Hannah Blythyn, 12,846, Conservativ...","Labour,",Hannah Blythyn,"12,846",...,None,None,None,None,None,None,None,None,None,None
6,Alyn & Deeside,North Wales,LAB HOLD,"65,884",39%,+4.60,"[Labour, , Jack Sargeant, 12,622, Conservative...","Labour,",Jack Sargeant,"12,622",...,None,None,None,None,None,None,None,None,None,None
7,Wrexham,North Wales,LAB HOLD,"53,181",43%,+3.00,"[Labour, , Lesley Griffiths, 8,452, Conservati...","Labour,",Lesley Griffiths,"8,452",...,None,None,None,None,None,None,None,None,None,None
8,Clwyd South,North Wales,LAB HOLD,"55,237",44%,+2.90,"[Labour, , Ken Skates, 10,448, Conservative, ,...","Labour,",Ken Skates,"10,448",...,None,None,None,None,None,None,None,None,None,None
9,Dwyfor Meirionnydd,Mid and West Wales,PC HOLD,"45,447",52%,+5.60,"[Plaid Cymru, , Mabon ap Gwynfor, 11,490, Cons...","Plaid Cymru,",Mabon ap Gwynfor,"11,490",...,None,None,None,None,None,None,None,None,None,None


## Clean df

> function needed here

In [431]:
# remove commas from turnout figure
df.turnout = df.turnout.str.replace(',', '_')
# remove% from turnout percentage figure
df.turnout_perc = df.turnout_perc.str.replace('%', '')

# remove commas from top four results score in votes
df.one_vote = df.one_vote.str.replace(',', '_')
df.two_vote = df.two_vote.str.replace(',', '_')
df.three_vote = df.three_vote.str.replace(',', '_')
df.four_vote = df.four_vote.str.replace(',', '_')

# remove trailing commas from top four party names
df.one_party = df.one_party.str[:-2]
df.two_party = df.two_party.str[:-2]
df.three_party = df.three_party.str[:-2]
df.four_party = df.four_party.str[:-2]

# drop the results column
df.drop("results", axis=1, inplace=True)

In [438]:
df.five_vote = df.five_vote.str.replace(',', '_')
df.six_vote = df.six_vote.str.replace(',', '_')
df.seven_vote = df.seven_vote.str.replace(',', '_')
df.eight_vote = df.eight_vote.str.replace(',', '_')
df.nine_vote = df.nine_vote.str.replace(',', '_')
df.ten_vote = df.ten_vote.str.replace(',', '_')
df.eleven_vote = df.eleven_vote.str.replace(',', '_')
df.twelve_vote = df.twelve_vote.str.replace(',', '_')

In [439]:
# all columns are strings

df.turnout = df.turnout.astype(int)
df.turnout_perc = df.turnout_perc.astype(int)
df.one_vote = df.one_vote.astype(int)
df.two_vote = df.two_vote.astype(int)
df.three_vote = df.three_vote.astype(int)
df.four_vote = df.four_vote.astype(int)
df.five_vote = df.five_vote.astype(int)
df.six_vote = df.six_vote.astype(int)
df.seven_vote = df.seven_vote.astype(int)
df.eight_vote = df.eight_vote.astype(int)
df.nine_vote = df.nine_vote.astype(int)
df.ten_vote = df.ten_vote.astype(int)
df.eleven_vote = df.eleven_vote.astype(int)
df.twelve_vote = df.twelve_vote.astype(int)


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [500]:
# export

df.to_csv('c_results.csv', index = False)

In [2]:
# import (if starting from here)

df = pd.read_csv('c_results.csv')

In [16]:
def my_const():
    
    print('Enter your constituency: ')
    x = input()

    winner = df[df.constituency == x].one_person.item()
    wparty = df[df.constituency == x].one_party.item()

    loser = df[df.constituency == x].two_person.item()
    lparty = df[df.constituency == x].two_party.item()
    
    winvote = df[df.constituency == x].one_vote.item()
    losevote = df[df.constituency == x].two_vote.item()
    margin = winvote - losevote
    
    # party hold / gain
    gh = df[df.constituency == x].result.item()[-4:].lower()
    if df[df.constituency == x].result.item()[:3] == 'LAB':
        ghp = 'Labour'
    if df[df.constituency == x].result.item()[:3] == 'CON':
        ghp = 'Tory'
    if df[df.constituency == x].result.item()[:3] == 'PC ':
        ghp = 'Plaid'
    
    # turnout (ranges from -3 to +5)
#     turnout_change = df[df.constituency == x].turnout_change.item()
#     if '-' in turnout_change:
#         to = 'down'
#     elif '+' in turnout_change:
#         to = 'up'
#     else:
#         to = 'unchanged'
    to_amt = df[df.constituency == x].turnout_perc.item()
    
    # region
    reg = df[df.constituency == x].region.item()
    
    # Write the par
    text = f"The victor in {x} was {winner} of {wparty}, who saw off {lparty}'s {loser} by {margin} votes for a {ghp} {gh}. Turnout here was {to_amt}%. {x} is part of the {reg} region which returned NEEDED to the Senedd."  
    
    return text


In [17]:
my_const()

Enter your constituency: 
Cardiff Central


"The victor in Cardiff Central was Jenny Rathbone of Labour, who saw off Liberal Democrat's Rodney Berman by 7640 votes for a Labour hold. Turnout here was 45%. Cardiff Central is part of the South Wales Central region which returned NEEDED to the Senedd."

In [ ]:
, {to} from the election five years ago ({turnout_change}%)

In [636]:
df[df.constituency == 'Cardiff Central']

,constituency,region,result,turnout,turnout_perc,turnout_change,one_party,one_person,one_vote,two_party,...,nine_vote,ten_party,ten_person,ten_vote,eleven_party,eleven_person,eleven_vote,twelve_party,twelve_person,twelve_vote
25,Cardiff Central,South Wales Central,LAB HOLD,64111,45,-1.10,Labour,Jenny Rathbone,13100,Liberal Democrat,...,151,"Socialist Party of Great Britain,",Brian Johnson,82,"Gwlad,",Alford Clement Thomas,65,None,None,None


In [641]:
df.sort_values('turnout_perc')

,constituency,region,result,turnout,turnout_perc,turnout_change,one_party,one_person,one_vote,two_party,...,nine_vote,ten_party,ten_person,ten_vote,eleven_party,eleven_person,eleven_vote,twelve_party,twelve_person,twelve_vote
18,Swansea East,South Wales West,LAB HOLD,60859,35,-0.30,Labour,Mike Hedges,12950,Plaid Cymru,...,None,None,None,None,None,None,None,None,None,None
36,Merthyr Tydfil & Rhymney,South Wales East,LAB HOLD,60787,35,-3.70,Labour,Dawn Bowden,13009,Plaid Cymru,...,None,None,None,None,None,None,None,None,None,None
29,Torfaen,South Wales East,LAB HOLD,64270,37,-0.80,Labour,Lynne Neagle,11572,Conservative,...,None,None,None,None,None,None,None,None,None,None
32,Newport East,South Wales East,LAB HOLD,60826,38,+1.00,Labour,John Griffiths,10899,Conservative,...,None,None,None,None,None,None,None,None,None,None
23,Cynon Valley,South Wales Central,LAB HOLD,52521,39,+0.90,Labour,Vikki Howells,11427,Plaid Cymru,...,None,None,None,None,None,None,None,None,None,None
6,Alyn & Deeside,North Wales,LAB HOLD,65884,39,+4.60,Labour,Jack Sargeant,12622,Conservative,...,None,None,None,None,None,None,None,None,None,None
30,Blaenau Gwent,South Wales East,LAB HOLD,51640,40,-1.80,Labour,Alun Davies,10226,Plaid Cymru,...,None,None,None,None,None,None,None,None,None,None
17,Swansea West,South Wales West,LAB HOLD,58587,41,+0.30,Labour,Julie James,11126,Conservative,...,189,None,None,None,None,None,None,None,None,None
37,Ogmore,South Wales West,LAB HOLD,58509,42,-1.00,Labour,Huw Irranca-Davies,12868,Plaid Cymru,...,None,None,None,None,None,None,None,None,None,None
20,Aberavon,South Wales West,LAB HOLD,52428,42,-0.20,Labour,David Rees,10505,Plaid Cymru,...,208,None,None,None,None,None,None,None,None,None


## Five regions

In [633]:
# scrape

main_list = []

r_list = [1, 6, 7, 8, 9]

for x in r_list:
    reg_dict = {}
    url = f'https://www.bbc.co.uk/news/election/2021/wales/regions/W1000000{x}'
    print(url)
    req = requests.get(url, headers = my_headers)
    soup = bs(req.content)

    for a in soup.find_all('h1', class_ = 'ssrcss-qozapo-StyledHeading e1fj1fc10'):
        place = a.text
        reg_dict['region'] = place

    for b in soup.find_all('span', class_ = 'ssrcss-1cuz2pt-TurnoutValue e1rrn48s6'):
        to = b.text
        reg_dict['turnout'] = to

    c_list = []
    for c in soup.find_all('span', class_ = 'ssrcss-9v20ri-StyledResult e1rrn48s8'):
        change = c.text
        c_list.append(change)
    reg_dict['registered'] = c_list[0]
    reg_dict['change'] = c_list[1]

    the_results = []
    for x in soup.find_all('div', class_ = 'ssrcss-1nfrpyt-GridContainer ez70f8k0'):
        results_list = []
        for y in x.find_all('h3'):
            party = y.text
            results_list.append(party)

            for z in x.find_all('span', class_ = 'ssrcss-ce9ddf-ResultValue e1k9l0jz2'):
                seats = z.text
                results_list.append(seats)

            for w in x.find_all('div', class_ = 'ssrcss-97fobn-Stack e1y4nx260'):
                numbers = w.text
                numbers = numbers.replace(u'\xa0', u' ')
                results_list.append(numbers)

            for b in x.find_all('ul', class_ = 'ssrcss-x3cdvy-ColumnList ei1exny2'):
                elected = b.text
                results_list.append(elected)

        the_results.append(results_list)

    reg_dict['final_results'] = the_results

    main_list.append(reg_dict)

https://www.bbc.co.uk/news/election/2021/wales/regions/W10000001
https://www.bbc.co.uk/news/election/2021/wales/regions/W10000006
https://www.bbc.co.uk/news/election/2021/wales/regions/W10000007
https://www.bbc.co.uk/news/election/2021/wales/regions/W10000008
https://www.bbc.co.uk/news/election/2021/wales/regions/W10000009


## make df

In [680]:
big_rows = []
for x in main_list:
    rows = []
    rows.append(x['region'])
    rows.append(x['turnout'])
    rows.append(x['registered'])
    rows.append(x['change'])
    
    rows.append(x['final_results'][0][0])
    rows.append(x['final_results'][0][1])
    rows.append(x['final_results'][0][3])
    
    rows.append(x['final_results'][1][0])
    rows.append(x['final_results'][1][1])
    try:
        rows.append(x['final_results'][1][3])
    except:
        pass
    
    rows.append(x['final_results'][2][0])
    rows.append(x['final_results'][2][1])
    try:
        rows.append(x['final_results'][2][3])
    except:
        pass
    
    rows.append(x['final_results'][3][0])
    rows.append(x['final_results'][3][1])
    try:
        rows.append(x['final_results'][3][3])
    except:
        pass


    big_rows.append(rows)


In [699]:
dfr = pd.DataFrame(big_rows)
dfr.columns = 'region turnout_pc reg_voters turnout_change party_one seats_one MS_one party_two seats_two MS_two party_three seats_three MS_three party_four seats_four'.split()
dfr

,region,turnout_pc,reg_voters,turnout_change,party_one,seats_one,MS_one,party_two,seats_two,MS_two,party_three,seats_three,MS_three,party_four,seats_four
0,North Wales,46%,"487,784",+2.90,Conservative,2,Mark IsherwoodSam Rowlands,Labour,1,Carolyn Thomas,Plaid Cymru,1,Llyr Gruffydd,Abolish the Welsh Assembly Party,0
1,Mid and West Wales,53%,"451,034",+2.20,Labour,2,Eluned MorganJoyce Watson,Plaid Cymru,1,Cefin Campbell,Liberal Democrat,1,Jane Dodds,Conservative,0
2,South Wales Central,48%,"531,912",+1.40,Conservative,2,Andrew RT DaviesJoel James,Plaid Cymru,2,Rhys ab OwenHeledd Fychan,Labour,0,Green,0,None
3,South Wales East,42%,"498,551",-0.20,Conservative,2,Laura Anne JonesNatasha Asghar,Plaid Cymru,2,Delyth JewellPeredur Owen Griffiths,Labour,0,Abolish the Welsh Assembly Party,0,None
4,South Wales West,44%,"417,676",+0.50,Conservative,2,Thomas GiffardAltaf Hussain,Plaid Cymru,2,Sioned WilliamsLuke Fletcher,Labour,0,Green,0,None


In [701]:
# remove problem characters
dfr.turnout_pc = dfr.turnout_pc.str.replace('%', '')
dfr.reg_voters = dfr.reg_voters.str.replace(',', '_')

# change type
dfr.turnout_pc = dfr.turnout_pc.astype(int)
dfr.reg_voters = dfr.reg_voters.astype(int)
dfr.seats_one = dfr.seats_one.astype(int)
dfr.seats_two = dfr.seats_two.astype(int)
dfr.seats_three = dfr.seats_three.astype(int)

In [703]:
dfr

,region,turnout_pc,reg_voters,turnout_change,party_one,seats_one,MS_one,party_two,seats_two,MS_two,party_three,seats_three,MS_three,party_four,seats_four
0,North Wales,46,487784,+2.90,Conservative,2,Mark IsherwoodSam Rowlands,Labour,1,Carolyn Thomas,Plaid Cymru,1,Llyr Gruffydd,Abolish the Welsh Assembly Party,0
1,Mid and West Wales,53,451034,+2.20,Labour,2,Eluned MorganJoyce Watson,Plaid Cymru,1,Cefin Campbell,Liberal Democrat,1,Jane Dodds,Conservative,0
2,South Wales Central,48,531912,+1.40,Conservative,2,Andrew RT DaviesJoel James,Plaid Cymru,2,Rhys ab OwenHeledd Fychan,Labour,0,Green,0,None
3,South Wales East,42,498551,-0.20,Conservative,2,Laura Anne JonesNatasha Asghar,Plaid Cymru,2,Delyth JewellPeredur Owen Griffiths,Labour,0,Abolish the Welsh Assembly Party,0,None
4,South Wales West,44,417676,+0.50,Conservative,2,Thomas GiffardAltaf Hussain,Plaid Cymru,2,Sioned WilliamsLuke Fletcher,Labour,0,Green,0,None


In [704]:
# export

dfr.to_csv('r_results.csv', index = False)